# Read Dataset

In [1]:
import pandas as pd
A = pd.read_csv("C:/Documents/DataScience/Restaurant_Reviews.tsv", sep="\t")

In [2]:
A

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
995,I think food should have flavor and texture an...,0
996,Appetite instantly gone.,0
997,Overall I was not impressed and would not go b...,0
998,"The whole experience was underwhelming, and I ...",0


In [3]:
A.drop_duplicates(inplace=True)

In [4]:
A.reset_index(drop=True, inplace=True)

In [5]:
A

,Review,Liked
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
991,I think food should have flavor and texture an...,0
992,Appetite instantly gone.,0
993,Overall I was not impressed and would not go b...,0
994,"The whole experience was underwhelming, and I ...",0


In [6]:
A["Liked"].value_counts()

1    499
0    497
Name: Liked, dtype: int64

# Cleaning The Data

In [7]:
from nltk.stem import PorterStemmer
from re import sub
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to C:\Users\Vijay
[nltk_data]     Mahajan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
Q = []
ps = PorterStemmer()

for i in range(0,A.shape[0]):
    message = sub(pattern='[^a-zA-Z]', repl=' ', string=A.Review[i]) #Cleaning special character from the message
    message = message.lower() #Converting the entire message into lower case
    words = message.split() # Tokenizing the review by words
    words = [word for word in words if word not in set(stopwords.words('english'))] #Removing the stop words
    words = [ps.stem(word) for word in words] #Stemming the words
    message = ' '.join(words) #Joining the stemmed words
    Q.append(message) #Building a corpus of messages

In [9]:
Q[0:15]

['wow love place',
 'crust good',
 'tasti textur nasti',
 'stop late may bank holiday rick steve recommend love',
 'select menu great price',
 'get angri want damn pho',
 'honeslti tast fresh',
 'potato like rubber could tell made ahead time kept warmer',
 'fri great',
 'great touch',
 'servic prompt',
 'would go back',
 'cashier care ever say still end wayyy overpr',
 'tri cape cod ravoli chicken cranberri mmmm',
 'disgust pretti sure human hair']

# Define our Y column and temp X column

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [11]:
X = cv.fit_transform(Q).toarray()
Y = A.Liked

# Split Data

In [12]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y, test_size=0.2, random_state=21)

# A NN

In [13]:
xtrain.shape

(796, 1565)

In [14]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

nn = Sequential()
nn.add(Dense(1000,input_dim=(1565)))
nn.add(Dropout(0.8))
nn.add(Dense(1000))
nn.add(Dropout(0.8))
nn.add(Dense(1,activation="sigmoid"))

In [15]:
nn.compile(loss="binary_crossentropy",metrics="accuracy")
nn.fit(xtrain,ytrain,epochs=100)

Epoch 1/100
25/25 [==============================] - 3s 63ms/step - loss: 0.6435 - accuracy: 0.6294
Epoch 2/100
25/25 [==============================] - 2s 63ms/step - loss: 0.4751 - accuracy: 0.7827
Epoch 3/100
25/25 [==============================] - 2s 63ms/step - loss: 0.3594 - accuracy: 0.8442
Epoch 4/100
25/25 [==============================] - 1s 42ms/step - loss: 0.3045 - accuracy: 0.8656
Epoch 5/100
25/25 [==============================] - 1s 41ms/step - loss: 0.2284 - accuracy: 0.9146
Epoch 6/100
25/25 [==============================] - 1s 49ms/step - loss: 0.1976 - accuracy: 0.9158
Epoch 7/100
25/25 [==============================] - 1s 42ms/step - loss: 0.1729 - accuracy: 0.9246
Epoch 8/100
25/25 [==============================] - 1s 46ms/step - loss: 0.1554 - accuracy: 0.9359
Epoch 9/100
25/25 [==============================] - 1s 46ms/step - loss: 0.1131 - accuracy: 0.9535
Epoch 10/100
25/25 [==============================] - 1s 47ms/step - loss: 0.1142 - accuracy: 0.9523

In [16]:
T =[]
for i in nn.predict(xtest):
    if(i<0.5):
        T.append(0)
    else:
        T.append(1)

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest,T)

0.725

In [18]:
def Review(df):
    x = cv.transform([df.lower()]).toarray()
    for i in nn.predict(x):
        if(i<0.5):
            print("Ohhh Shit! Negetive Review")
        else:
            print("Good! Positive Review")

In [19]:
Rev = "I did not like the soup"

In [20]:
Review(Rev)

Ohhh Shit! Negetive Review


In [21]:
r = " Food is teasty"
Review(r)

Good! Positive Review


In [22]:
r = "The food was absolutely wonderful, from preparation to presentation, very pleasing."
Review(r)

Good! Positive Review


In [23]:
r = "Loved this food"
Review(r)

Good! Positive Review


In [24]:
r = "Loved"
Review(r)

Good! Positive Review


In [25]:
r = " hate this food"
Review(r)

Ohhh Shit! Negetive Review


In [26]:
r = "Restarunt is owesome"
Review(r)

Good! Positive Review
